In [9]:
# libraries
import pandas as pd
import mysql.connector

In [10]:
def check_duplicates(df):
    df2 = df.duplicated()
    for i in df2:
        if i == True:
            key = 1
            break
        else:
            key = 0
    return key

In [11]:
# importing data from csv files
book_detail_table = pd.read_csv('d:/2/bookData.csv')
book_detail_table['Isbn'].fillna('null', inplace = True)
book_detail_table['score'].fillna(0, inplace = True)
#book_detail_table
#check_duplicates(book_detail_table)

In [12]:
translator_table = pd.read_csv('d:/2/translator.csv')
#(translator_table)
#(check_duplicates(translator_table))

In [13]:
translator_page_table = pd.read_csv('d:/2/translator_info.csv')
translator_page_table['info'].fillna('null', inplace = True)
translator_page_table.drop_duplicates(subset="id", keep = 'first', inplace=True)
#(translator_page_table)
#(check_duplicates(translator_page_table))

In [14]:
price_history_table = pd.read_csv('d:/2/price-history.csv')
#(price_history_table)
#(check_duplicates(price_history_table))

In [15]:
publisher_table = pd.read_csv('d:/2/publisher.csv')
#(publisher_table)
#(check_duplicates(publisher_table))

In [16]:
format_table = pd.read_csv('d:/2/format.csv')
format_table.rename(columns = {'Unnamed: 0': 'id'}, inplace = True)
format_table.loc[format_table.shape[0]] = [-1, 'null']
#(format_table)
#(check_duplicates(format_table))

In [17]:
cover_type_table = pd.read_csv('d:/2/cover_type.csv')
cover_type_table.rename(columns = {'Unnamed: 0': 'id'}, inplace = True)
cover_type_table.loc[cover_type_table.shape[0]] = [-1, 'null']
#(cover_type_table)
#(check_duplicates(cover_type_table))

In [18]:
book_summary_table = pd.read_csv('d:/2/BookSummaryData.csv')
#(book_summary_table)
#(check_duplicates(book_summary_table))

In [19]:
columns_i_need = book_detail_table.site_id
book_summary_table.rename(columns = {'site_index':'site_id'}, inplace = True)
book_summary_new_table = pd.merge(columns_i_need, book_summary_table, on = 'site_id', how = 'left')
#book_summary_new_table

In [20]:
book_summary_new_table.drop_duplicates(subset="site_id", keep = 'first', inplace=True)
book_summary_new_table['summary'].fillna('null', inplace = True)
book_summary_new_table.reset_index(inplace = True, drop = True)
#book_summary_new_table

In [21]:
reward_table = pd.read_csv('d:/2/award.csv')
#(reward_table)
#(check_duplicates(reward_table))

In [22]:
summary_id = set(list(book_summary_new_table.site_id))
reward_id = set(list(reward_table.site_index))
diff = list(reward_id - summary_id)
for i in diff:
    reward_table.drop(reward_table.loc[reward_table['site_index'] == i].index, inplace=True)

reward_table.reset_index(inplace = True, drop = True)

In [23]:
book_tag_table = pd.read_csv('d:/2/bookTagsData.csv')
#(book_tag_table)
#(check_duplicates(book_tag_table))

In [24]:
summary_id = set(list(book_summary_new_table.site_id))
b_tag_id = set(list(book_tag_table.site_index))
diff = list(b_tag_id - summary_id)
for i in diff:
    book_tag_table.drop(book_tag_table.loc[book_tag_table['site_index'] == i].index, inplace=True)

book_tag_table.reset_index(inplace = True, drop = True)
#book_tag_table

In [25]:
tag_table = pd.read_csv('d:/2/tag.csv')
tag_table.rename(columns = {'Unnamed: 0': 'id'}, inplace = True)
#(tag_table)
#(check_duplicates(tag_table))

In [26]:
book_veneration_table = pd.read_csv('d:/2/book_veneration.csv')
book_veneration_table['Prise_Writer'].fillna('Unknow', inplace=True)
#(book_veneration_table)
#(check_duplicates(book_veneration_table))

In [27]:
summary_id = set(list(book_summary_new_table.site_id))
veneration_id = set(list(book_veneration_table.site_index))
difference = list(veneration_id - summary_id)
for i in difference:
    book_veneration_table.drop(book_veneration_table.loc[book_veneration_table['site_index'] == i].index, inplace=True)

book_veneration_table.reset_index(inplace = True, drop = True)
#book_veneration_table

In [28]:
writer_table = pd.read_csv('d:/2/writer.csv')
#(writer_table)
#(check_duplicates(writer_table))

In [29]:
d = dict()
for i in range(book_detail_table.shape[0]):
    d[str(book_detail_table.code[i])] = book_detail_table.site_id[i]
lis = list()
for w in range(writer_table.shape[0]):
     lis.append(d.get(str(writer_table.book_id[w])))
writer_table['site_id'] = lis

writer_new_table = writer_table.drop(columns=['book_id'], inplace = False)
writer_new_table.drop_duplicates(keep = 'first', inplace=True)
writer_new_table.reset_index(inplace = True, drop = True)

In [30]:
writer_page_table = pd.read_csv('d:/2/writer_info.csv')
writer_page_table['info'].fillna('null', inplace = True)
writer_page_table.drop_duplicates(subset="id", keep = 'first', inplace=True)
#(writer_page_table)
#(check_duplicates(writer_page_table))

In [31]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="a*7EE166B4")
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE BookUtopia")

In [32]:
mydb = mysql.connector.connect(
   host="localhost",
  user="root",
  password="a*7EE166B4",
  database="BookUtopia")
mycursor = mydb.cursor()

In [33]:
mycursor.execute('create table book_summary(site_id int primary key, summary text)')

In [34]:
for i in range(book_summary_new_table.shape[0]):
    sql = 'insert into book_summary(site_id, summary) values (%s, %s)'
    record = (int(book_summary_new_table.site_id[i]), book_summary_new_table.summary[i])
    mycursor.execute(sql, record)
mydb.commit()

In [35]:
mycursor.execute('create table publisher(id int primary key, name varchar(255))')

In [36]:
for i in range(publisher_table.shape[0]):
    sql = 'insert into publisher (id, name) values (%s, %s)'
    record = (int(publisher_table['id'][i]), publisher_table.name[i])
    mycursor.execute(sql, record)
mydb.commit()  

In [37]:
mycursor.execute('create table format(id tinyint primary key, name varchar(255))')

In [38]:
for i in range(format_table.shape[0]):
    sql = 'insert into format (id, name) values(%s, %s)'
    record = (int(format_table['id'][i]), format_table['size'][i])
    mycursor.execute(sql, record)
mydb.commit()

In [39]:
mycursor.execute('create table cover_type(id tinyint primary key, name varchar(255))')

In [40]:
for i in range(cover_type_table.shape[0]):
    sql = 'insert into cover_type(id, name) values(%s, %s)'
    record = (int(cover_type_table['id'][i]), cover_type_table.cover_material[i])
    mycursor.execute(sql, record)
mydb.commit()

In [41]:
mycursor.execute('create table rewards(id int auto_increment primary key, site_id int, reward text,\
                FOREIGN KEY (site_id) REFERENCES book_summary(site_id))')

In [42]:
for i in range(reward_table.shape[0]):
    sql = 'insert into rewards (site_id, reward) values(%s, %s)'
    record = (int(reward_table.site_index[i]), reward_table.award[i])
    mycursor.execute(sql, record)
mydb.commit()

In [43]:
mycursor.execute('create table tags(id int primary key, name varchar(255))')

In [44]:
for i in range(tag_table.shape[0]):
    sql = 'insert into tags (id, name) values(%s, %s)'
    record = (int(tag_table['id'][i]), tag_table.tag[i])
    mycursor.execute(sql, record)

mydb.commit()

In [45]:
mycursor.execute('create table book_tag(id int auto_increment primary key, site_id int, tag_id int,\
FOREIGN KEY (site_id) REFERENCES book_summary(site_id),\
FOREIGN KEY (tag_id) REFERENCES tags(id))')

In [46]:
for i in range(book_tag_table.shape[0]):
    sql = 'insert into book_tag (site_id, tag_id) values (%s, %s)'
    record = (int(book_tag_table.site_index[i]), int(book_tag_table.tag[i]))
    mycursor.execute(sql, record)
mydb.commit()

In [47]:
mycursor.execute('create table veneration(id int auto_increment primary key, site_id int, English_Quote text, Persian_Quote text, prise_writer varchar(255),\
FOREIGN KEY (site_id) REFERENCES book_summary(site_id))')

In [48]:
for i in range(book_veneration_table.shape[0]):
    sql = 'insert into veneration (site_id, English_Quote, Persian_Quote, prise_writer) values (%s, %s, %s, %s)'
    record = (int(book_veneration_table.site_index[i]), book_veneration_table.English_Quote[i],
              book_veneration_table.Persian_Quote[i], book_veneration_table.Prise_Writer[i])
    mycursor.execute(sql, record)
mydb.commit()

In [49]:
mycursor.execute('create table writer_page (writer_id int primary key, name varchar(255), about text)')

In [50]:
for i in range(writer_page_table.shape[0]):
    sql = 'insert into writer_page(writer_id, name, about) values(%s, %s, %s)'
    record = (int(writer_page_table['id'][i]), writer_page_table.name[i], writer_page_table['info'][i])
    mycursor.execute(sql, record)
mydb.commit()

In [51]:
mycursor.execute('create table writer(id int auto_increment primary key, writer_id int, site_id int,\
FOREIGN KEY (writer_id) REFERENCES writer_page(writer_id),\
FOREIGN KEY (site_id) REFERENCES book_summary(site_id))')

In [52]:
for i in range(writer_new_table.shape[0]):
    sql = 'insert into writer(writer_id, site_id) values (%s, %s)'
    record = (int(writer_new_table.writer_id[i]), int(writer_new_table.site_id[i]))
    mycursor.execute(sql, record)
mydb.commit()

In [53]:
mycursor.execute('create table translator_page(translator_id int primary key, name varchar(255), about text)')

In [54]:
for i in range(translator_page_table.shape[0]):
    sql = 'insert into translator_page(translator_id, name, about) values (%s, %s, %s)'
    record = (int(translator_page_table['id'][i]), translator_page_table.name[i], translator_page_table['info'] [i])
    mycursor.execute(sql, record)
mydb.commit()

In [55]:
mycursor.execute('create table book_detail(\
book_id int primary key,\
site_id int not null,\
Persian_title varchar(225),\
English_title varchar(225),\
publisher_id int,\
score decimal(5,2),\
ISBN varchar(128),\
format_id tinyint,\
page_number int,\
cover_id tinyint,\
edition smallint,\
solar_publication_year int,\
ad_publication_year int,\
book_language varchar(128),\
delivery_time varchar(128),\
stock_status varchar(128),\
paper_type varchar(128),\
FOREIGN KEY (site_id) REFERENCES book_summary(site_id),\
FOREIGN KEY (format_id) REFERENCES format(id),\
FOREIGN KEY (cover_id) REFERENCES cover_type(id),\
FOREIGN KEY (publisher_id) REFERENCES publisher(id))')

In [56]:
for i in range(book_detail_table.shape[0]):
    sql = 'insert into book_detail values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    record = (int(book_detail_table['code'][i]), int(book_detail_table.site_id[i]), book_detail_table.fa_title[i],
              book_detail_table.en_title[i], int(book_detail_table.publisher_id[i]), float(book_detail_table.score[i]),
              book_detail_table.Isbn[i], int(book_detail_table['size'][i]), int(book_detail_table['pages'][i]),
              int(book_detail_table.cover_material[i]), int(book_detail_table.print_series[i]), 
              int(book_detail_table.publication_per_date[i]), int(book_detail_table.publication_ad_date[i]),
              book_detail_table.language[i], book_detail_table.earliest_send_time[i], book_detail_table.presence[i], book_detail_table.paper_type[i])
    mycursor.execute(sql, record)
mydb.commit()

In [57]:
mycursor.execute('create table translator(id int auto_increment primary key, translator_id int, book_id int,\
FOREIGN KEY (translator_id) REFERENCES translator_page(translator_id),\
FOREIGN KEY (book_id) REFERENCES book_detail(book_id))')

In [58]:
for i in range(translator_table.shape[0]):
    sql = 'insert into translator (translator_id, book_id) values (%s, %s)'
    record = (int(translator_table.translator_id[i]), int(translator_table.book_id[i]))
    mycursor.execute(sql, record)
mydb.commit()

In [59]:
mycursor.execute("CREATE TABLE Price_History (id int auto_increment primary key,\
book_id int, price int, discount int, date datetime,\
FOREIGN KEY (book_id) REFERENCES book_detail(book_id))")

In [60]:
for i in range(price_history_table.shape[0]):
    sql = 'insert into Price_history (book_id, price, discount, date) values (%s, %s, %s, %s)'
    record = (int(price_history_table.book_id[i]), int(price_history_table.price[i]), int(price_history_table.discount[i]), price_history_table.date[i])
    mycursor.execute(sql, record)
mydb.commit()